In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

import numpy as np
from functools import partial
from statistics import mean, stdev
import time


import utils
import generators
import bases
import representations
import eigensolvers
import const as C

def get_circle_obs_krylov(initial_state=0, num_col=200):
    return utils.generate_krylov(generators.angle_evolution, generators.full_state_observable, initial_state, num_col)

def get_zero_norm_ratio(V):
    N = V.shape[1]
    n = N
    for k in range(N):
        if np.linalg.norm(V[:,k]) < C.POWER_EPS:
            n -= 1
    return n/N

def run(U, basis=bases.fourier_basis, degree=100, representation=representations.EDMD_matrix_representation, n_run=10):
    K, V, L = representation(U, degree=degree, basis=basis)
    results = []
    times = []
    for i in range(n_run):
        st = time.time()
        new_L, new_V = eigensolvers.power_orthogonal_QR_algorithm(K, partial(basis, degree=degree))
        et = time.time()
        r = get_zero_norm_ratio(new_V)
        m = (et-st)/60
        results.append(r)
        times.append(m)
        print(f"i={i} new V: {r} time {m}[m]")
    print("ratio mean {:>5.2f}, std {:>5.2f}".format(mean(results), stdev(results)))
    print("time  mean {:>5.2f}, std {:>5.2f}".format(mean(times), stdev(times)))
    return results, times

def pure_inv_run(U, basis=bases.fourier_basis, degree=100, representation=representations.EDMD_matrix_representation, n_run=10):
    K, V, L = representation(U, degree=degree, basis=basis)
    results = []
    times = []
    for i in range(n_run):
        st = time.time()
        new_L, new_V = eigensolvers.QR_algorithm_with_inverse_iteration(K)
        et = time.time()
        r = get_zero_norm_ratio(new_V)
        m = (et-st)/60
        results.append(r)
        times.append(m)
        print(f"i={i} new V: {r} time {m}[m]")
    print("ratio mean {:>5.2f}, std {:>5.2f}".format(mean(results), stdev(results)))
    print("time  mean {:>5.2f}, std {:>5.2f}".format(mean(times), stdev(times)))
    return results, times

In [2]:
Ut = np.expand_dims(get_circle_obs_krylov(), axis=1)
print(f"Ut shape{Ut.shape}")

Ut shape(200, 1)


### No skip

In [3]:
ns_results, ns_times = run(Ut)

i=0 new V: 0.5223880597014925 time 20.159166479110716[m]
i=1 new V: 0.6616915422885572 time 22.505152014891305[m]
i=2 new V: 0.5223880597014925 time 21.000311799844106[m]
i=3 new V: 0.5621890547263682 time 21.600652901331582[m]
i=4 new V: 0.5223880597014925 time 20.09588698546092[m]
i=5 new V: 0.5223880597014925 time 21.18584463596344[m]
i=6 new V: 0.29850746268656714 time 14.858158032099405[m]
i=7 new V: 0.29850746268656714 time 14.957205363114674[m]
i=8 new V: 0.48258706467661694 time 19.967465913295747[m]
i=9 new V: 0.42786069651741293 time 18.575653449694315[m]
mean  0.48, std  0.11
mean 19.49, std  2.64


In [10]:
print("ratio mean {:>5.2f}[%], std {:>5.2f}".format(mean(ns_results), stdev(ns_results)))
print("time  mean {:>5.2f}[m], std {:>5.2f}".format(mean(ns_times), stdev(ns_times)))

ratio mean  0.48[%], std  0.11
time  mean 19.49[m], std  2.64


### Stop if it finds zero vector

In [13]:
s_results, s_times = run(Ut)

i=0 new V: 0.42786069651741293 time 18.555441665649415[m]
i=1 new V: 0.48258706467661694 time 19.568171413739524[m]
i=2 new V: 0.6965174129353234 time 22.470659550031026[m]
i=3 new V: 0.42786069651741293 time 18.557164851824442[m]
i=4 new V: 0.34328358208955223 time 16.375960433483122[m]
i=5 new V: 0.5223880597014925 time 19.90091269016266[m]
i=6 new V: 0.6019900497512438 time 22.05832331577937[m]
i=7 new V: 0.40298507462686567 time 17.49313413699468[m]
i=8 new V: 0.48258706467661694 time 19.665427366892498[m]
i=9 new V: 0.582089552238806 time 21.124796601136527[m]
ratio mean  0.50, std  0.11
time  mean 19.58, std  1.94


### Pure inverse iteration

In [3]:
pure_inv_run(Ut)

i=0 new V: 1.0 time 0.22982688347498575[m]
i=1 new V: 1.0 time 0.21183103322982788[m]
i=2 new V: 1.0 time 0.22463393608729046[m]
i=3 new V: 1.0 time 0.2087887684504191[m]
i=4 new V: 1.0 time 0.20808204809824626[m]
i=5 new V: 1.0 time 0.2064029534657796[m]
i=6 new V: 1.0 time 0.20931435426076253[m]
i=7 new V: 1.0 time 0.21095928351084392[m]
i=8 new V: 1.0 time 0.20856939951578776[m]
i=9 new V: 1.0 time 0.21178683042526245[m]
ratio mean  1.00, std  0.00
time  mean  0.21, std  0.01


([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [0.22982688347498575,
  0.21183103322982788,
  0.22463393608729046,
  0.2087887684504191,
  0.20808204809824626,
  0.2064029534657796,
  0.20931435426076253,
  0.21095928351084392,
  0.20856939951578776,
  0.21178683042526245])